In [ ]:
#TODO: Please change the path to the MIMIC-IV dataset
MIMIC_IV_PATH = 'YOUR PATH HERE'

# Load Data

In [ ]:
import tqdm
import pandas as pd

In [ ]:
patients = pd.read_csv(f'{MIMIC_IV_PATH}/patients.csv.gz', compression='gzip')

omr_records = pd.read_csv(f'{MIMIC_IV_PATH}/omr.csv.gz', compression='gzip')
admission_records = pd.read_csv(f'{MIMIC_IV_PATH}/admissions.csv.gz', compression='gzip')
pharmcy_records = pd.read_csv(f'{MIMIC_IV_PATH}/pharmacy.csv.gz', compression='gzip')
prescriptions_records = pd.read_csv(f'{MIMIC_IV_PATH}/prescriptions.csv.gz', compression='gzip')

diagnoses_records = pd.read_csv(f'{MIMIC_IV_PATH}/diagnoses_icd.csv.gz', compression='gzip')
diagnose_label = pd.read_csv(f'{MIMIC_IV_PATH}/d_icd_diagnoses.csv.gz', compression='gzip')

# Process Data

### Basic information

In [7]:
patients_ids = list(set(patients['subject_id'].tolist()))

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [8]:
admission_records_sub = admission_records[admission_records['subject_id'].isin(patients_ids)]

11897


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [10]:
qualified_patients_ids = admission_records_sub.subject_id.unique()
qualified_patients_ids = qualified_patients_ids[:10000]

In [11]:
ages = []
genders = []
dods = []

for i in range(admission_records_sub.shape[0]):
    subject_id = admission_records_sub.iloc[i]['subject_id']
    gender = patients[patients['subject_id'] == subject_id]['gender'].values[0]
    achor_age = patients[patients['subject_id'] == subject_id]['anchor_age'].values[0]
    achor_year = patients[patients['subject_id'] == subject_id]['anchor_year'].values[0]
    current_year = admission_records_sub.iloc[i]['admittime'].split('-')[0]
    current_age = achor_age + (int(current_year) - achor_year)
    ages.append(current_age)
    genders.append("Female" if gender == 'F' else "Male")
    dods.append(patients[patients['subject_id'] == subject_id]['dod'].values[0])

[52, 52, 52, 52, 19, 72, 72, 25, 55, 57, 34, 20, 53, 74, 34, 86, 32, 32, 32, 20, 20, 18, 22, 57, 53, 57, 57, 53, 80, 77, 78, 78, 76, 75, 78, 64, 48, 46, 46, 55, 55, 30, 33, 29, 43, 47, 43, 46, 89, 89, 89, 91, 89, 89, 35, 71, 55, 86, 46, 73, 84, 90, 89, 76, 71, 71, 76, 71, 72, 76, 76, 74, 76, 77, 76, 71, 73, 76, 76, 73, 76, 76, 76, 76, 76, 59, 19, 62, 56, 57, 54, 57, 59, 63, 53, 61, 58, 53, 64, 87, 92, 87, 83, 81, 80, 81, 82, 37, 34, 71, 72, 68, 72, 72, 38, 27, 77, 81, 81, 80, 85, 85, 81, 82, 86, 90, 90, 86, 53, 81, 26, 28, 37, 82, 21, 39, 88, 56, 59, 58, 32, 32, 32, 33, 66, 66, 40, 40, 38, 58, 56, 56, 48, 53, 48, 54, 52, 51, 53, 52, 52, 55, 71, 79, 71, 72, 73, 74, 73, 73, 72, 73, 69, 72, 73, 71, 73, 79, 64, 79, 69, 89, 75, 75, 71, 75, 65, 65, 75, 70, 73, 73, 41, 75, 75, 75, 72, 74, 74, 74, 44, 91, 88, 86, 89, 91, 94, 90, 88, 58, 57, 58, 57, 61, 61, 67, 91, 31, 33, 47, 47, 47, 34, 64, 64, 63, 60, 38, 82, 85, 85, 83, 85, 84, 85, 85, 85, 84, 83, 85, 78, 65, 68, 66, 73, 72, 66, 64, 64, 64,

In [ ]:
admission_records_sub['age'] = ages
admission_records_sub['gender'] = genders
admission_records_sub['dod'] = dods

### Merge Records

In [13]:
diagnoses_records_sub = diagnoses_records[diagnoses_records['subject_id'].isin(qualified_patients_ids)]
diagnoses_records_sub = diagnoses_records_sub.merge(diagnose_label[['icd_code', 'long_title']], left_on='icd_code', right_on='icd_code')
print(diagnoses_records_sub.subject_id.nunique())
diagnoses_records_sub.drop(columns=['icd_code', "icd_version"], inplace=True)
diagnoses_records_sub.rename(columns={"long_title": "diagnosis"}, inplace=True)

9998


,subject_id,hadm_id,seq_num,diagnosis
0,10000032,22595853,1,Portal hypertension
1,10000032,22595853,2,Other ascites
2,10000032,22595853,3,Cirrhosis of liver without mention of alcohol
3,10000032,22595853,4,Unspecified viral hepatitis C without hepatic ...
4,10000032,22595853,5,"Chronic airway obstruction, not elsewhere clas..."


In [14]:
unique_diagnoses = diagnoses_records_sub['diagnosis'].unique()

10637


In [15]:
omr_records_sub = omr_records[omr_records['subject_id'].isin(qualified_patients_ids)]

6777


,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15


In [17]:
blood_pressures = []
weights = []
heights = []
bmi = []

for i in range(diagnoses_records_sub.shape[0]):
    subject_id = diagnoses_records_sub.iloc[i]['subject_id']
    seq_num = diagnoses_records_sub.iloc[i]['seq_num']
    omr_records_sub_temp = omr_records_sub[(omr_records_sub['subject_id'] == subject_id) & (omr_records_sub['seq_num'] == seq_num)]
    blood_pressure_temp  = omr_records_sub_temp[omr_records_sub_temp['result_name'] == 'Blood Pressure']
    weight_temp = omr_records_sub_temp[omr_records_sub_temp['result_name'] == 'Weight (Lbs)']
    height_temp = omr_records_sub_temp[omr_records_sub_temp['result_name'] == 'Height (Inches)']
    bmi_temp = omr_records_sub_temp[omr_records_sub_temp['result_name'] == 'BMI (kg/m2)']
    blood_pressures.append(str(blood_pressure_temp['result_value'].values[0]) if blood_pressure_temp.shape[0] > 0 else "unknown")
    weights.append(str(weight_temp['result_value'].values[0]) if weight_temp.shape[0] > 0 else "unknown")
    heights.append(str(height_temp['result_value'].values[0]) if height_temp.shape[0] > 0 else "unknown")
    bmi.append(str(bmi_temp['result_value'].values[0]) if bmi_temp.shape[0] > 0 else "unknown")

diagnoses_records_sub['weight'] = weights
diagnoses_records_sub['height'] = heights
diagnoses_records_sub['bmi'] = bmi
diagnoses_records_sub['blood_pressure'] = blood_pressures

In [ ]:
pharmcy_records_sub = pharmcy_records[pharmcy_records['subject_id'].isin(qualified_patients_ids)]

In [ ]:
prescriptions_records_sub = prescriptions_records[prescriptions_records['subject_id'].isin(qualified_patients_ids)]

In [ ]:
drugs = []
prod_strengths = []
for i in range(diagnoses_records_sub.shape[0]):
    subject_id = diagnoses_records_sub.iloc[i]['subject_id']
    hadm_id = diagnoses_records_sub.iloc[i]['hadm_id']
    prescriptions_records_sub_temp = prescriptions_records_sub[(prescriptions_records_sub['subject_id'] == subject_id) & (prescriptions_records_sub['hadm_id'] == hadm_id)]
    drugs_temp = prescriptions_records_sub_temp['drug'].tolist()
    prod_strengths_temp = prescriptions_records_sub_temp['prod_strength'].tolist()
    drugs_temp = [x for x in drugs_temp if x == x]
    prod_strengths_temp = [x for x in prod_strengths_temp if x == x]
    
    medications_temp = ""
    for j in range(len(drugs_temp)):
        if j < len(prod_strengths_temp): 
            medications_temp += drugs_temp[j] + " (" + prod_strengths_temp[j] + "), "
        else:
            medications_temp += drugs_temp[j] + ", "
    medications_temp = medications_temp[:-2]
    drugs.append(medications_temp if len(medications_temp) > 0 else "unknown")

diagnoses_records_sub['medications'] = drugs

In [ ]:
mereged_records = diagnoses_records_sub.merge(admission_records_sub, left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'], how='left')

In [26]:
mereged_records = mereged_records[mereged_records["seq_num"] == 1]
mereged_records.to_csv('./merged_records.csv', index=False)

In [44]:
patient_records = []
for i in tqdm.tqdm(range(len(qualified_patients_ids))):
    patient_id = qualified_patients_ids[i]
    temp = {}
    patient_records_temp = mereged_records[mereged_records['subject_id'] == patient_id]
    if patient_records_temp.shape[0] == 0:
        continue
    temp['id'] = str(patient_id)
    temp["gender"] = patient_records_temp.iloc[0]["gender"]
    temp['race'] = patient_records_temp.iloc[0]['race'].lower()
    dod = patient_records_temp.iloc[0]['dod']
    temp['day_of_death'] = dod if dod == dod else "unknown"
    
    records = []
    for i in range(patient_records_temp.shape[0]):
        patient_record = {}
        patient_record['current_age'] = str(patient_records_temp.iloc[i]['age'])
        marital_status = patient_records_temp.iloc[i]['marital_status']
        patient_record['marital_status'] = marital_status.lower() if marital_status == marital_status else "unknown"
        patient_record['weight'] = patient_records_temp.iloc[i]['weight']
        patient_record['height'] = patient_records_temp.iloc[i]['height']
        patient_record['bmi'] = patient_records_temp.iloc[i]['bmi']
        patient_record['blood_pressure'] = patient_records_temp.iloc[i]['blood_pressure']
        patient_record['medications'] = patient_records_temp.iloc[i]['medications']
        patient_record['admit_time'] = patient_records_temp.iloc[i]['admittime'].split(' ')[0]
        patient_record['admit_type'] = patient_records_temp.iloc[i]['admission_type'].lower()
        patient_record['insurance'] = patient_records_temp.iloc[i]['insurance'].lower()
        patient_record['language'] = patient_records_temp.iloc[i]['language'].lower()
        admit_location = patient_records_temp.iloc[i]['admission_location']
        patient_record['admit_location'] = admit_location.lower() if admit_location == admit_location else "unknown"
        patient_record['discharge_time'] = patient_records_temp.iloc[i]['dischtime'].split(' ')[0]
        discharge_location = patient_records_temp.iloc[i]['discharge_location']
        patient_record['discharge_location'] = discharge_location.lower() if discharge_location == discharge_location else "unknown"
        patient_record['diagnosis'] = patient_records_temp.iloc[i]['diagnosis']
        records.append(patient_record)
        
    records = sorted(records, key=lambda x: x['admit_time'])
    
    target = records[-1]
    if target["weight"] == "unknown":
        target["weight"] = records[0]["weight"]
    if target["height"] == "unknown":
        target["height"] = records[0]["height"]
    if target["bmi"] == "unknown":
        target["bmi"] = records[0]["bmi"]
    records = records[:-1]
        
    temp['records'] = records
    temp["target"] = target
    patient_records.append(temp)

100%|██████████| 10000/10000 [00:07<00:00, 1289.35it/s]


In [47]:
import json
with open('./patient_records.json', 'w') as f:
    json.dump(patient_records, f)

In [48]:
id_ = []
gender = []
race = []
age = []
day_of_death = []
height = []
weight = []
bmi = []
blood_pressure = []
marital_status = []
number_of_records = []
insurance = []
language = []
admit_time = []
admit_type = []
discharge_time = []
admit_location = []
discharge_location = []

for patient in patient_records:
    id_.append(patient['id'])
    gender.append(patient['gender'])
    race.append(patient['race'])
    day_of_death.append(patient['day_of_death'])
    number_of_records.append(len(patient['records']))
    target_record = patient['target']
    age.append(target_record['current_age'])
    height.append(target_record['height'])
    weight.append(target_record['weight'])
    bmi.append(target_record['bmi'])
    blood_pressure.append(target_record['blood_pressure'])
    marital_status.append(target_record['marital_status'])
    insurance.append(target_record['insurance'])
    language.append(target_record['language'])
    admit_type.append(target_record['admit_type'])
    admit_time.append(target_record['admit_time'])
    admit_location.append(target_record['admit_location'])
    discharge_time.append(target_record['discharge_time'])
    length_of_stay = (pd.to_datetime(target_record['discharge_time']) - pd.to_datetime(target_record['admit_time'])).days
    discharge_location.append(target_record['discharge_location'])
    
data = {'id': id_, 'gender': gender, 'race': race, 'age': age, 'height': height, 'weight': weight, 'bmi': bmi, 'blood_pressure': blood_pressure, 'marital_status': marital_status, 'number_of_records': number_of_records, 'insurance': insurance, 'language': language, 'admit_type': admit_type, 'admit_time': admit_time, 'admit_location': admit_location, 'discharge_time': discharge_time, 'discharge_location': discharge_location}

df = pd.DataFrame(data)
cleaned_df = df.dropna()
cleaned_df.to_csv('./cleaned_data.csv', index=False) 

In [76]:
cleaned_df['discharge_location'] = cleaned_df['discharge_location'].apply(lambda x: x if x in ['home', 'died'] else 'other')

['other' 'home' 'died']


discharge_location
other    5911
home     3599
died      488
Name: count, dtype: int64

In [77]:
admit_cata = {
  "emergency": ["ew emer.", "urgent", "direct emer."],
  "elective": ["elective"],
  "planned": ["eu observation", "observation admit", "surgical same day admission", "ambulatory observation", "direct observation"]
}

cleaned_df['admit_type'] = cleaned_df['admit_type'].apply(lambda x: 'emergency' if x in admit_cata['emergency'] else ('elective' if x in admit_cata['elective'] else 'planned'))

['ew emer.' 'eu observation' 'observation admit'
 'surgical same day admission' 'ambulatory observation' 'urgent'
 'direct observation' 'direct emer.' 'elective']
['emergency' 'planned' 'elective']


In [78]:
cleaned_df['admit_type'] = cleaned_df['admit_type'].apply(lambda x: x if x in ['emergency', 'urgent', 'elective'] else 'other')

# Split Data

In [ ]:
cleaned_df = cleaned_df.dropna()

cleaned_df_home = cleaned_df[cleaned_df['discharge_location'] == 'home']
cleaned_df_other = cleaned_df[cleaned_df['discharge_location'] == 'other']
cleaned_df_died = cleaned_df[cleaned_df['discharge_location'] == 'died'] 
sample_home = cleaned_df_home.sample(1000, random_state=42)
sample_other = cleaned_df_other.sample(1000, random_state=42)
sample_died = cleaned_df_died.sample(1000, random_state=42)
sample_died = cleaned_df_died
test_subset = pd.concat([sample_home, sample_other, sample_died])

In [73]:
test_subset.to_csv('test_subset_data.csv', index=False)

In [82]:
other_df = cleaned_df.drop(test_subset.index)

In [85]:
other_home = other_df[other_df['discharge_location'] == 'home']
other_other = other_df[other_df['discharge_location'] == 'other']
other_died = other_df[other_df['discharge_location'] == 'died']
train_subset = pd.concat([other_home.sample(1500, random_state=42), other_other.sample(1500, random_state=42), other_died.sample(1500, random_state=42)])
train_subset.to_csv('train_subset_data.csv', index=False)